##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoGraph: Fácil Controle para Fluxos de Grafos

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r1/guide/autograph.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar em Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/autograph.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizar no GitHub</a>
  </td>
</table>

[AutoGraph](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/) helps you write complicated graph code using normal Python. Behind the scenes, AutoGraph automatically transforms your code into the equivalent [TensorFlow graph code](https://www.tensorflow.org/r1/guide/graphs). AutoGraph already supports much of the Python language, and that coverage continues to grow. For a list of supported Python language features, see the [Autograph capabilities and limitations](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md).

## Configuração

Importar TensorFlow, AutoGraph, e alguns módulos de suporte:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

In [0]:
layers = tf.keras.layers

import numpy as np
import matplotlib.pyplot as plt

Vamos habilitar a  [execução ansiosa](https://www.tensorflow.org/r1/guide/eager) para demonstração de propósitos, mas o AutoGraph trabalha em ambos, [execução de grafo](https://www.tensorflow.org/r1/guide/graphs) ambientes:

Nota: O código convertido do AutoGraph foi projetado para ser executado durante a execução do grafo. Quando a execução ansiosa estiver ativada, use grafos explícitos (como mostra este exemplo) ou `tf.contrib.eager.defun`.

## Converter automaticamente o fluxo de controle Python

O AutoGraph converterá grande parte da linguagem Python no código de construção de grafo TensorFlow equivalente.

Nota: Em aplicativos reais, o lote é essencial para o desempenho. O melhor código para converter em AutoGraph é o código em que o fluxo de controle é decidido no nível _batch_. Ao tomar decisões no nível _exemplo_ individual, você deve indexar e agrupar os exemplos para manter o desempenho enquanto aplica a lógica do fluxo de controle.

O AutoGraph converte uma função como:

In [0]:
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0.0
  return x

Para uma função que usa construção de grafo:

In [0]:
print(tf.autograph.to_code(square_if_positive))

O código escrito para execução rápida pode ser executado em um `tf.Graph` com os mesmos resultados, mas com os benefícios da execução do grafo:

In [0]:
print('Eager results: %2.2f, %2.2f' % (square_if_positive(tf.constant(9.0)),
                                       square_if_positive(tf.constant(-9.0))))

Gere uma versão gráfica e chame-a:

In [0]:
tf_square_if_positive = tf.autograph.to_graph(square_if_positive)

with tf.Graph().as_default():
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
  g_out1 = tf_square_if_positive(tf.constant( 9.0))
  g_out2 = tf_square_if_positive(tf.constant(-9.0))
  with tf.Session() as sess:
    print('Graph results: %2.2f, %2.2f\n' % (sess.run(g_out1), sess.run(g_out2)))

O AutoGraph suporta instruções Python comuns como `while`,` for`, `if`,` break` e `return`, com suporte para aninhamento. Compare esta função com a versão gráfica complicada exibida nos seguintes blocos de código:

In [0]:
# Continue in a loop
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s

print('Eager result: %d' % sum_even(tf.constant([10,12,15,20])))

tf_sum_even = tf.autograph.to_graph(sum_even)

with tf.Graph().as_default(), tf.Session() as sess:
    print('Graph result: %d\n\n' % sess.run(tf_sum_even(tf.constant([10,12,15,20]))))

In [0]:
print(tf.autograph.to_code(sum_even))

## tf.function

Use o decorador `tf.function`:

In [0]:
@tf.function(
    experimental_autograph_options=tf.autograph.experimental.Feature.EQUALITY_OPERATORS)
def fizzbuzz(i, n):
  while i < n:
    msg = ''
    if i % 3 == 0:
      msg += 'Fizz'
    if i % 5 == 0:
      msg += 'Buzz'
    if msg == '':
      msg = tf.as_string(i)
    tf.print(msg)
    i += 1
  return i

with tf.Graph().as_default():
  final_i = fizzbuzz(tf.constant(10), tf.constant(16))
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
  with tf.Session() as sess:
    sess.run(final_i)



## Examplos

Vamos demonstrar alguns recursos úteis da linguagem Python.


### Assert

O AutoGraph pode converter automaticamente a instrução `assert` do Python para o código `tf.Assert` equivalente:

In [0]:
@tf.function(
    experimental_autograph_options=(
        tf.autograph.experimental.Feature.ASSERT_STATEMENTS,
        tf.autograph.experimental.Feature.EQUALITY_OPERATORS))
def inverse(x):
  assert x != 0.0, 'Do not pass zero!'
  return 1.0 / x

with tf.Graph().as_default(), tf.Session() as sess:
  try:
    print(sess.run(inverse(tf.constant(0.0))))
  except tf.errors.InvalidArgumentError as e:
    print('Got error message:\n    %s' % e.message)

### Print

Opcionalmente, você pode usar a função `print` do Python no grafo, quando combinada com o gerenciamento automático de dependências de controle do` tf.function`:

In [0]:
@tf.function(
    experimental_autograph_options=tf.autograph.experimental.Feature.BUILTIN_FUNCTIONS)
def count(n):
  i = 0
  while i < n:
    print(i)
    i += 1
  return n

with tf.Graph().as_default(), tf.Session() as sess:
    sess.run(count(tf.constant(5)))

### Listas

Anexar às listas em loops (as operações da lista de tensores são criadas automaticamente):

In [0]:
@tf.function(
    experimental_autograph_options=tf.autograph.experimental.Feature.LISTS)
def arange(n):
  z = tf.TensorArray(tf.int32, size=0, dynamic_size=True)

  for i in tf.range(n):
    z.append(i)

  return z.stack()


with tf.Graph().as_default(), tf.Session() as sess:
    print(sess.run(arange(tf.constant(10))))

### Fluxo de controle aninhado

In [0]:
@tf.function(
    experimental_autograph_options=tf.autograph.experimental.Feature.EQUALITY_OPERATORS)
def nearest_odd_square(x):
  if x > 0:
    x = x * x
    if x % 2 == 0:
      x = x + 1
  return x

with tf.Graph().as_default():
  with tf.Session() as sess:
    print(sess.run(nearest_odd_square(tf.constant(4))))
    print(sess.run(nearest_odd_square(tf.constant(5))))
    print(sess.run(nearest_odd_square(tf.constant(6))))

### Laço While

In [0]:
@tf.function
def square_until_stop(x, y):
  while x < y:
    x = x * x
  return x

with tf.Graph().as_default():
  with tf.Session() as sess:
    print(sess.run(square_until_stop(tf.constant(4), tf.constant(100))))

### Laço For

In [0]:
@tf.function(
    experimental_autograph_options=tf.autograph.experimental.Feature.LISTS)
def squares(nums):

  result = tf.TensorArray(tf.int64, size=0, dynamic_size=True)
  
  for num in nums:
    result.append(num * num)

  return result.stack()

with tf.Graph().as_default():
  with tf.Session() as sess:
    print(sess.run(squares(tf.constant(np.arange(10)))))

### Break

In [0]:
@tf.function
def argwhere_cumsum(x, threshold):
  current_sum = 0.0
  idx = 0
  for i in tf.range(len(x)):
    idx = i
    if current_sum >= threshold:
      break
    current_sum += x[i]
  return idx

N = 10
with tf.Graph().as_default():
  with tf.Session() as sess:
    idx = argwhere_cumsum(tf.ones(N), tf.constant(float(N/2)))
    print(sess.run(idx))

## Interoperação com `tf.Keras`

É fácil integrar `tf.autograph` com `tf.keras`.

### Funções sem estado

Para funções sem estado, como `collatz` mostrado abaixo, a maneira mais fácil de incluí-los em um modelo keras é envolvê-los como uma camada usando `tf.keras.layers.Lambda`.

In [0]:
import numpy as np

@tf.function(
    experimental_autograph_options=(
        tf.autograph.experimental.Feature.ASSERT_STATEMENTS,
        tf.autograph.experimental.Feature.EQUALITY_OPERATORS,
        ))
def collatz(x):
  x = tf.reshape(x,())
  assert x > 0
  n = tf.convert_to_tensor((0,))
  while x != 1:
    n += 1
    if x % 2 == 0:
      x = x // 2
    else:
      x = 3 * x + 1

  return n

with tf.Graph().as_default():
  model = tf.keras.Sequential([
    tf.keras.layers.Lambda(collatz, input_shape=(1,), output_shape=())
  ])

  result = model.predict(np.array([6171]))
  print(result)

### Customização Avançada de Modelos

<!--TODO(markdaoust) link to full examples  or these referenced models.-->

Para subclasses de modelos Keras, a maneira mais fácil é converter o método `call '. Consulte o [Guia TensorFlow Keras] (https://tensorflow.org/r1/guide/keras#build_advanced_models) para obter detalhes sobre como desenvolver essas classes.

Aqui está um exemplo simples da técnica [profundidade da rede estocástica] (https://arxiv.org/abs/1603.09382):

In [0]:
# `K` is used to check if we're in train or test mode.
K = tf.keras.backend

class StochasticNetworkDepth(tf.keras.Sequential):
  def __init__(self, layers, pfirst=1.0, plast=0.5,**kwargs):
    self.pfirst = pfirst
    self.plast = plast
    super(StochasticNetworkDepth, self).__init__(layers,**kwargs)

  def build(self, input_shape):
    self.depth = len(self.layers)
    self.plims = np.linspace(self.pfirst, self.plast, self.depth + 1)[:-1]
    super(StochasticNetworkDepth, self).build(input_shape.as_list())

  def call(self, inputs):
    training = tf.cast(K.learning_phase(), dtype=bool)
    if not training:
      count = self.depth
      return super(StochasticNetworkDepth, self).call(inputs), count

    p = tf.random_uniform((self.depth,))

    keeps = (p <= self.plims)
    x = inputs

    count = tf.reduce_sum(tf.cast(keeps, tf.int32))
    for i in range(self.depth):
      if keeps[i]:
        x = self.layers[i](x)

    # return both the final-layer output and the number of layers executed.
    return x, count

StochasticNetworkDepth.call = tf.autograph.to_graph(StochasticNetworkDepth.call)

Vamos tentar com dados em forma de mnist:

In [0]:
train_batch = np.random.randn(64, 28, 28, 1).astype(np.float32)

Crie uma pilha simples de camadas `conv`, no modelo de profundidade estocástica:

In [0]:
with tf.Graph().as_default() as g:
  model = StochasticNetworkDepth(
      [
        layers.Conv2D(filters=16, activation=tf.nn.relu,
                  kernel_size=(3, 3), padding='same')
        for n in range(20)
      ],
      pfirst=1.0, plast=0.5
  )

  model.build(tf.TensorShape((None, None, None, 1)))

  init = tf.global_variables_initializer()

Agora teste-o para garantir que ele se comporte conforme o esperado nos modos de trem e teste:

In [0]:
# Use an explicit session here so we can set the train/test switch, and
# inspect the layer count returned by `call`
with tf.Session(graph=g) as sess:
  init.run()

  for phase, name in enumerate(['test','train']):
    K.set_learning_phase(phase)
    result, count = model(tf.convert_to_tensor(train_batch, dtype=tf.float32))

    result1, count1 = sess.run((result, count))
    result2, count2 = sess.run((result, count))

    delta = (result1 - result2)
    print(name, "sum abs delta: ", abs(delta).mean())
    print("    layers 1st call: ", count1)
    print("    layers 2nd call: ", count2)
    print()

## Exemplo avançado: um loop de treinamento no grafo

A seção anterior mostrou que o AutoGraph pode ser usado dentro de camadas e modelos de Keras. Os modelos Keras também podem ser usados no código de AutoGraph.

Como o fluxo de controle de gravação no AutoGraph é fácil, a execução de um ciclo de treinamento em um grafo TensorFlow também deve ser fácil.

Este exemplo mostra como treinar um modelo Keras simples no MNIST com todo o processo de treinamento - carregamento de lotes, cálculo de gradientes, atualização de parâmetros, cálculo da precisão da validação e repetição até a convergência - é realizado no grafo.

### Download dos dados

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

### Definição do Modelo

In [0]:
def mlp_model(input_shape):
  model = tf.keras.Sequential((
      tf.keras.layers.Dense(100, activation='relu', input_shape=input_shape),
      tf.keras.layers.Dense(100, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')))
  model.build()
  return model


def predict(m, x, y):
  x = tf.to_float(x) / 255.0
  y = tf.one_hot(tf.squeeze(y), 10)
  y_p = m(tf.reshape(x, (-1, 28 * 28)))
  losses = tf.keras.losses.categorical_crossentropy(y, y_p)
  l = tf.reduce_mean(losses)
  accuracies = tf.keras.metrics.categorical_accuracy(y, y_p)
  accuracy = tf.reduce_mean(accuracies)
  return l, accuracy


def fit(m, x, y, opt):
  l, accuracy = predict(m, x, y)
  # Autograph automatically adds the necessary `tf.control_dependencies` here.
  # (Without them nothing depends on `opt.minimize`, so it doesn't run.)
  # This makes it much more like eager-code.
  opt.minimize(l)
  return l, accuracy


def setup_mnist_data(is_training, batch_size):
  if is_training:
    ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    ds = ds.shuffle(batch_size * 10)
  else:
    ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  ds = ds.repeat()
  ds = ds.batch(batch_size)
  return ds


### Definição do laço de treinamento

In [0]:
def train(train_ds, test_ds, learning_rate, max_steps):
  m = mlp_model((28 * 28,))
  opt = tf.train.AdamOptimizer(learning_rate)

  train_losses = tf.TensorArray(tf.float32, size=0, dynamic_size=True, element_shape=())
  test_losses = tf.TensorArray(tf.float32, size=0, dynamic_size=True, element_shape=())
  train_accuracies = tf.TensorArray(tf.float32, size=0, dynamic_size=True, element_shape=())
  test_accuracies = tf.TensorArray(tf.float32, size=0, dynamic_size=True, element_shape=())

  # This entire training loop will be run in-graph.
  i = tf.constant(0)
  for (train_x, train_y), (test_x, test_y) in tf.data.Dataset.zip((train_ds, test_ds)):
    step_train_loss, step_train_accuracy = fit(m, train_x, train_y, opt)
    step_test_loss, step_test_accuracy = predict(m, test_x, test_y)
    if i % 50 == 0:
      print('Step', i, 'train loss:', step_train_loss, 'test loss:',
            step_test_loss, 'train accuracy:', step_train_accuracy,
            'test accuracy:', step_test_accuracy)
    train_losses.append(step_train_loss)
    test_losses.append(step_test_loss)
    train_accuracies.append(step_train_accuracy)
    test_accuracies.append(step_test_accuracy)

    i += 1
    if i >= max_steps:
      break

  # We've recorded our loss values and accuracies
  # to a list in a graph with AutoGraph's help.
  # In order to return the values as a Tensor,
  # we need to stack them before returning them.
  return (train_losses.stack(), test_losses.stack(),
          train_accuracies.stack(), test_accuracies.stack())
  
train = tf.autograph.to_graph(
    train,
    experimental_optional_features=(
        tf.autograph.experimental.Feature.LISTS,
        tf.autograph.experimental.Feature.BUILTIN_FUNCTIONS,
        tf.autograph.experimental.Feature.EQUALITY_OPERATORS,
        tf.autograph.experimental.Feature.AUTO_CONTROL_DEPS))


Agora construa o grafo e execute o ciclo de treinamento:

In [0]:
with tf.Graph().as_default() as g:
  learning_rate = 0.005
  max_steps=500

  train_ds = setup_mnist_data(True, 50)
  test_ds = setup_mnist_data(False, 1000)
  (train_losses, test_losses, train_accuracies,
   test_accuracies) = train(train_ds, test_ds, learning_rate, max_steps)

  init = tf.global_variables_initializer()

with tf.Session(graph=g) as sess:
  sess.run(init)
  (train_losses, test_losses, train_accuracies,
   test_accuracies) = sess.run([train_losses, test_losses, train_accuracies,
                                test_accuracies])


In [0]:
plt.title('MNIST train/test losses')
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.xlabel('Training step')
plt.ylabel('Loss')
plt.show()
plt.title('MNIST train/test accuracies')
plt.plot(train_accuracies, label='train accuracy')
plt.plot(test_accuracies, label='test accuracy')
plt.legend(loc='lower right')
plt.xlabel('Training step')
plt.ylabel('Accuracy')
plt.show()